# GITHUB REPOSITORY SETUP:
    - Github repository: jays-codes/team24
    - Description: Main branch, developent branch, and release branch. 
    - Each team member forked the repository (all branches), and worked (push/pull changes) on the development branch.

# DATA COLLECTION & ELT: 

- Performed ETL (Extract, Load, Transform): 
    - Extracted the following dataset, saved to SQLite database and Github repo: 
        - TTC Streetcar Delay, FY2023 and YTD-09-2024 (https://open.toronto.ca/dataset/ttc-streetcar-delay-data/): directly extracted to Github. 
    
    - Created the following datasets, saved to SQLite database and/or Python dataframes and Github repository:  
        - Ontario Public Holiday, 2023 and 2024 (https://excelnotes.com/holidays-ontario-2023/ and https://excelnotes.com/holidays-ontario-2024): no file available, only information online. We manually created the datasets in .csv and saved to Github. 
        - Line route (https://www.ttc.ca/routes-and-schedules/listroutes/streetcar): no file available, only information online. We manually created the datasets in .csv and saved to Github.  
    
    - Loaded: 
        - Loaded the data to SQLite database 
        - From SQLite database, loaded to Python Panda dataframe. 
   
    - Transformed: 
        - Predictors:
            - Day (3 categories - weekday, weekend, holiday)
            - Holiday (12 categorical values)
            - Time (4 categories - morning, afternoon, evening, night)
            - Peak time (3 categories – peak time morning, peak time afternoon, no)
            - Line (4 categories – regular, special, night, others (bus substitute))
            - Location (1679 unique locations)
            - Bound (4 categorical – north, south, west, east)
            - Vehicle (1012 vehicles)
            - Incident category (13 categories)
        
        - Response variable:
            - Min Delay category (3 categories – short, medium, long)
   
The tasks were done by: Jay Menarco. See codes below.


In [1]:
#Read from SQLite database and load to a pandas dataframe
import os
import sqlite3
import pandas as pd


In [2]:
# Function to load data from SQLite database
def load_from_db(db_name, table_name):
    conn = sqlite3.connect(db_name)
    query = f'SELECT * FROM {table_name}'
    df = pd.read_sql(query, conn)
    conn.close()

    return df

In [3]:

# Check if __file__ is defined
if '__file__' in globals():
    # Get the directory two levels up from the current script
    base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
else:
    # Fallback for environments where __file__ is not defined
    base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
    
    # Construct the relative path to the database
db_name = os.path.join(base_dir, 'data', 'streetcardelaydb.db')
table_name = 'Streetcar_Delay_Data'  # Replace with your table name

# Check if the database file exists
if not os.path.exists(db_name):
    raise FileNotFoundError(f"Database file not found: {db_name}")

# Load data from SQLite database
df = load_from_db(db_name, table_name)

# Convert Incident_Date to datetime
df['incident_date'] = pd.to_datetime(df['incident_date'])

# Load Date table to get isHoliday, isWeekend, and isEndOfMth columns
date_table_name = 'Date'  # Replace with your date table name
date_df = load_from_db(db_name, date_table_name)
date_df['date'] = pd.to_datetime(date_df['date'])

# Merge Date table with Streetcar_Delay_Data table on incident_date
df = df.merge(date_df[['date', 'isHoliday', 'isWeekend', 'isEndOfMth']], left_on='incident_date', right_on='date', how='left')
df.drop(columns=['date'], inplace=True)


# Load Line table to get lineType
line_table_name = 'Line'  # Replace with your line table name
line_df = load_from_db(db_name, line_table_name)

# Merge the dataframes on lineId
df = df.merge(line_df[['lineId', 'lineType']], left_on='line', right_on='lineId', how='left')

#for lineType that has not been set, set it to 4
df['lineType'] = df['lineType'].fillna(4)

# Load Delay table to get delayType
delay_table_name = 'Delay'  # Replace with your delay table name
delay_df = load_from_db(db_name, delay_table_name)

    # Function to determine delayType
def get_delay_type(min_delay):
    for _, row in delay_df.iterrows():
        if row['delayFrom'] <= min_delay <= row['delayTo']:
            return row['delayId']
    return None

# Apply the function to determine delayType
df['delayType'] = df['min_delay'].apply(get_delay_type)

# Display the DataFrame
df.head()


,incident_date,line,incident_time,day_of_week,location,incident,min_delay,min_gap,bound,vehicle,isHoliday,isWeekend,isEndOfMth,lineId,lineType,delayType
0,2023-01-01,505,02:40,Sunday,BROADVIEW AND GERRARD,Held By,15,25,W,4460,1,1,0,505,2.0,2
1,2023-01-01,504,02:52,Sunday,KING AND BATHURST,Cleaning - Unsanitary,10,20,W,4427,1,1,0,504,2.0,2
2,2023-01-01,504,02:59,Sunday,KING AND BATHURST,Held By,25,35,E,4560,1,1,0,504,2.0,3
3,2023-01-01,510,05:38,Sunday,SPADINA AND DUNDAS,Security,15,30,S,4449,1,1,0,510,2.0,2
4,2023-01-01,506,06:35,Sunday,OSSINGTON STATION,Security,10,20,None,8706,1,1,0,506,2.0,2


# FEATURE ENGINEERING 

Tasks were done by: Ly Nguyen 

In [4]:
# Display exploratory statistics
df.describe()

,incident_date,min_delay,min_gap,isHoliday,isWeekend,isEndOfMth,lineType,delayType
count,21742,21742.000000,21742.000000,21742.000000,21742.000000,21742.000000,21742.000000,21742.000000
mean,2023-11-17 14:32:47.730659584,18.185494,27.249885,0.027504,0.282771,0.032978,2.017708,2.133014
min,2023-01-01 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2023-06-13 00:00:00,8.000000,16.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,2023-11-24 00:00:00,10.000000,20.000000,0.000000,0.000000,0.000000,2.000000,2.000000
75%,2024-04-15 00:00:00,15.000000,24.000000,0.000000,1.000000,0.000000,2.000000,2.000000
max,2024-09-30 00:00:00,960.000000,968.000000,1.000000,1.000000,1.000000,4.000000,3.000000
std,NaN,37.272247,39.340414,0.163552,0.450356,0.178582,0.261982,0.561122


In [5]:
# Assess datatype and any NULL values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21742 entries, 0 to 21741
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   incident_date  21742 non-null  datetime64[ns]
 1   line           21698 non-null  object        
 2   incident_time  21742 non-null  object        
 3   day_of_week    21742 non-null  object        
 4   location       21741 non-null  object        
 5   incident       21742 non-null  object        
 6   min_delay      21742 non-null  int64         
 7   min_gap        21742 non-null  int64         
 8   bound          19127 non-null  object        
 9   vehicle        21742 non-null  object        
 10  isHoliday      21742 non-null  int64         
 11  isWeekend      21742 non-null  int64         
 12  isEndOfMth     21742 non-null  int64         
 13  lineId         21574 non-null  object        
 14  lineType       21742 non-null  float64       
 15  delayType      2174

Approach to handle NULL and missing values: as we have plenty of data (>20,000 observations) and only less than 2,000 NULL values, we can remove NULL values without significant impact.

In [6]:
# Convert incident_time to datetime
df['incident_time'] = pd.to_datetime(df['incident_time'], format='%H:%M').dt.time

# Define function to categorize time
def categorize_time(time):
    if time >= pd.to_datetime('06:30').time() and time <= pd.to_datetime('09:30').time():
        return 1
    elif time >= pd.to_datetime('10:00').time() and time <= pd.to_datetime('14:00').time():
        return 2
    elif time >= pd.to_datetime('16:30').time() and time <= pd.to_datetime('19:00').time():
        return 3
    else:
        return 4

# Apply function to create new column "timeType"
df['timeType'] = df['incident_time'].apply(categorize_time)


# Clean and factorize columns: location, bound, incident, vehicle, day_of_week

# Define a helper function
def clean_and_factorize(df, column_name, valid_values=None):
    # Remove leading and trailing spaces
    df[column_name] = df[column_name].str.strip()
    
    # Drop null values
    df = df.dropna(subset=[column_name])
    
    # If valid values are provided, filter the dataframe
    if valid_values:
        df = df[df[column_name].isin(valid_values)]
    
    # Assign unique numeric values
    df[f"{column_name}_id"] = pd.factorize(df[column_name])[0] + 1
    return df

# Assuming your DataFrame is named df

# Apply the function to each column
df = clean_and_factorize(df, 'location')
df = clean_and_factorize(df, 'incident')
df = clean_and_factorize(df, 'vehicle')
df = clean_and_factorize(df, 'day_of_week')
df = clean_and_factorize(df, 'bound', valid_values=['N', 'S', 'W', 'E'])

# Display the DataFrame
df.head()


C:\Users\DELL\AppData\Local\Temp\ipykernel_23036\3883107428.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{column_name}_id"] = pd.factorize(df[column_name])[0] + 1


,incident_date,line,incident_time,day_of_week,location,incident,min_delay,min_gap,bound,vehicle,...,isEndOfMth,lineId,lineType,delayType,timeType,location_id,incident_id,vehicle_id,day_of_week_id,bound_id
0,2023-01-01,505,02:40:00,Sunday,BROADVIEW AND GERRARD,Held By,15,25,W,4460,...,0,505,2.0,2,4,1,1,1,1,1
1,2023-01-01,504,02:52:00,Sunday,KING AND BATHURST,Cleaning - Unsanitary,10,20,W,4427,...,0,504,2.0,2,4,2,2,2,1,1
2,2023-01-01,504,02:59:00,Sunday,KING AND BATHURST,Held By,25,35,E,4560,...,0,504,2.0,3,4,2,1,3,1,2
3,2023-01-01,510,05:38:00,Sunday,SPADINA AND DUNDAS,Security,15,30,S,4449,...,0,510,2.0,2,4,3,3,4,1,3
5,2023-01-01,501,08:52:00,Sunday,LAKE SHORE AND THIRTY,Cleaning - Unsanitary,10,20,W,8015,...,0,501,2.0,2,1,5,2,6,1,1


In [7]:
# List of columns to drop
columns_to_drop = ['incident_date', 'line', 'min_gap', 'lineId', 'isEndOfMth', 'min_delay', 'location', 'bound', 'incident', 'vehicle','incident_time','day_of_week']

# Drop the specified columns
df_cleaned = df.drop(columns=columns_to_drop)

# Verify if any NULL values remain, and assess dtype: 
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19109 entries, 0 to 21741
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isHoliday       19109 non-null  int64  
 1   isWeekend       19109 non-null  int64  
 2   lineType        19109 non-null  float64
 3   delayType       19109 non-null  int64  
 4   timeType        19109 non-null  int64  
 5   location_id     19109 non-null  int64  
 6   incident_id     19109 non-null  int64  
 7   vehicle_id      19109 non-null  int64  
 8   day_of_week_id  19109 non-null  int64  
 9   bound_id        19109 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 1.6 MB


In [8]:
# Visualize the cleaned dataset: 
df_cleaned.head()

,isHoliday,isWeekend,lineType,delayType,timeType,location_id,incident_id,vehicle_id,day_of_week_id,bound_id
0,1,1,2.0,2,4,1,1,1,1,1
1,1,1,2.0,2,4,2,2,2,1,1
2,1,1,2.0,3,4,2,1,3,1,2
3,1,1,2.0,2,4,3,3,4,1,3
5,1,1,2.0,2,1,5,2,6,1,1


In [9]:
# Verify that the values are as expected (e.g., min and max)
df_cleaned.describe()

,isHoliday,isWeekend,lineType,delayType,timeType,location_id,incident_id,vehicle_id,day_of_week_id,bound_id
count,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000,19109.000000
mean,0.026951,0.282328,2.015961,2.123136,3.137527,566.637134,5.543880,337.969281,4.080904,1.997540
std,0.161943,0.450144,0.252902,0.559111,1.091400,725.204579,2.966374,355.149453,1.990754,1.003471
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,2.000000,2.000000,2.000000,90.000000,3.000000,98.000000,2.000000,1.000000
50%,0.000000,0.000000,2.000000,2.000000,4.000000,247.000000,5.000000,209.000000,4.000000,2.000000
75%,0.000000,1.000000,2.000000,2.000000,4.000000,704.000000,8.000000,408.000000,6.000000,2.000000
max,1.000000,1.000000,4.000000,3.000000,4.000000,3197.000000,14.000000,1578.000000,7.000000,4.000000
